In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


# ARIMA: Autoregressive Integrated Moving Average

## By Team Fourier

This notebook will provide an introduction into ARIMA, short for autoregressive integrated moving average. ARIMA models aim to model time series data such as the weather, stock prices, or product sales over a period of time. The focus will be on weather data which has a seasonal aspect to it. The notebook will move into SARIMAX, short for seasonal autoregressive integrated moving average, a model that allows for seasonal periodic behavior. The notebook will rely heavily on the statsmodels Python package.

## Learning Objectives

1. Provide some insight into the power of time series analysis and moving average models as opposed to traditional function fitting techniques such as linear regression.

2. Provide a brief introduction to techniques in order to determine important characteristics of your data (stationarity, seasonality) that will influence decisions when choosing a model.

3. Give an introduction into statsmodels ARIMA and SARIMAX capabilities. In real applications, these functions are what you would use to implement an ARIMA model yourself.

4. Provide some examples of visualization, an important skill for data scientists.

5. A brief example of using grid search in order to choose optimal hyperparameters.

## Part 1: Smoothing

We can think of timeseries data as the following form:

\begin{align*}
X_{t} = m_{t} + Z_{t}
\end{align*}

This formulation of timeseries data means there is an underlying model function $m(y)$ and residuals at each observation point: $Z_{t}$. The purpose of timeseries analysis is to predict these $Z_{t}$'s using a combination of the previous $p$ or $q$ $Z_{t}$'s or the observations themselves. In this case, having an $m_{t}$ is inconvenient for our model as we only want to predict $Z_{t}$'s. Therefore, we can perform a technique called smoothing.

There are 2 main ways to perform smoothing:

1. Perform a linear or quadratic fit on the data, then simply reduce the model to $Y_{t} = X_{t} - f(t)$ which becomes $Y_{t} = Z_{t}$ if $f(t) = m_{t}$. 

2. Perform smoothing using a moving average fit.

Derive $m_{t}$ using the formula: 

\begin{align*}
m_{t} = \frac{1}{2q+1} + \sum_{j=-q}^{q} X_{t+j}
\end{align*}

Then obtain the new observations: $Y_{t} = X_{t} - m_{t}$.

### Question 1: Perform smoothing using regression

Implement smoothing using regression to find a linear or quadratic fit on the data.

In [ ]:
def generate_timeseries(deg = 'linear', samples = 100):
    if deg == 'linear':
        w = np.random.uniform(2, 5)
        X = np.arange(samples).astype(np.float32)
        X *= w
    elif deg == 'quadratic':
        w = np.random.uniform(0.03, 0.08, size=(2,1))
        X = np.vstack((np.arange(samples), np.square(np.arange(samples)))).astype(np.float32).T
        X = X @ w
    elif deg == 'sinusoidal':
        period = 20
        w = np.random.uniform(50, 100)
        X = np.sin(np.arange(samples).astype(np.float32) * 2 * np.pi / period)
        X *= w
    else:
        print('only support deg = linear, quadratic, sinusoidal')
        
    X = np.random.normal(X, samples / 10)
    X = X.reshape((X.shape[0], 1))
    return X


def plot_timeseries(ts, label):
    plt.plot(ts, label=label)
    plt.xlabel('time')
    plt.ylabel('observations')


def smoothing_via_regression(X, fit="linear"):
    plot_timeseries(X, 'Timeseries Data Before Smoothing')
    
    #TODO: implement a linear or quadratic regression fit
    Y = ... #output Y = X - m
    m = ... #fitted model, should be the same shape as X
    
    
    #YOUR CODE HERE
    if fit == 'linear':
        time = ...
        w = ...
        m = ...
        Y = ...
        
        
    elif fit == 'quadratic':
        time = ...
        w = ...
        m = ...
        Y = ...
    #END YOUR CODE
        
    else:
        print('fit must be one of "linear" or "quadratic"')
        
    plot_timeseries(Y, 'Timeseries Data After Model Fit')
    plot_timeseries(m, 'Fitted Model')
    plt.legend()
    plt.title("Smoothing using %s regression" % fit)
    plt.show()


### Question 2: Perform smoothing using moving average

Implement smoothing using a moving average to approximate $m_{t}$.

In [ ]:
def smoothing_via_moving_average(X, q=1):
    plot_timeseries(X, 'Timeseries Data Before Smoothing')
    
    assert(X.shape[0] > 2 * q + 1)
    
    #TODO: implement a moving average fit using the parameter q
    
    #YOUR CODE HERE
    #try using np.convolve(mode="same"), don't worry about weird values on the boundaries
    
    m = None #fitted model, same shape as X
    Y = None #output Y = X - m
    
    #END YOUR CODE
        
    plot_timeseries(Y, 'Timeseries Data After Model Fit')
    plot_timeseries(m, 'Fitted Model')
    plt.legend()
    plt.title("Smoothing using moving average with parameter q=%d" % q)
    plt.show()
    

In [ ]:
X_linear = generate_timeseries(deg='linear')
smoothing_via_regression(X_linear, fit='linear')
smoothing_via_moving_average(X_linear, q=3)


X_quad = generate_timeseries(deg='quadratic')
smoothing_via_regression(X_quad, fit='quadratic')
smoothing_via_moving_average(X_quad, q=3)

X_sin = generate_timeseries(deg='sinusoidal')
smoothing_via_regression(X_sin, fit='linear')
smoothing_via_regression(X_sin, fit='quadratic')
smoothing_via_moving_average(X_sin, q=3)

### Question 3: Test Smoothing on a Real Dataset

In [ ]:
data = pd.read_csv('data/TSLA.csv')
time = np.array(data['Date'])
price = np.array(data['Close'])

plt.title('Tesla Stock Price')
plt.plot(time, price)
plt.xticks(time[::50])
plt.xlabel('Days since November 19, 2019')
plt.ylabel('Price')
plt.show()

price = price.reshape((price.shape[0], 1))

smoothing_via_regression(price, fit='linear')
smoothing_via_moving_average(price, q=5)
smoothing_via_regression(price, fit='quadratic')
smoothing_via_moving_average(price, q=5)

### Observations:
1. Which smoothing performed better using the synthetic data?

2. Which smoothing performed better using the real Tesla data? Why?

3. Try using different values of q, what happens when the value of q becomes very large. What happens when it is too small?

## Part 2: Stationarity and Differencing

Smoothing is one way to enforce stationarity on the timeseries data. What is stationarity?  
Stationarity is the idea that the data is the same no matter where you look. Formally, that is the following equation is the same for the joint distributions given any $t$, $q$, $n$:

\begin{align*}
P(X_{t}, ..., X_{t+n}) = P(X_{q}, ..., X_{q+n})
\end{align*}

For data, you can assess this qualitatively: No obvious trend, and no seasonality. If the data, however, has some sort of trend or seasonality, we can get rid of them via differencing.

### Question 1: Trend Differencing

If our data has a trend, meaning it can be expressed as:

\begin{align*}
X_{t} = m_{t} + Z_{t}
\end{align*}

Differencing is a technique in which we can create a new timeseries:

\begin{align*}
Y_{t} = X_{t} - X_{t-1}
\end{align*}

If our trend is linear, for instance, we can achieve the following result using differencing:

\begin{align*}
m_{t} = at + b\\
X_{t} = m_{t} + Z_{t}\\
X_{t-1} = m_{t-1} + Z_{t-1}\\
Y = X_{t} - X_{t-1} = a + Z_{t} - Z_{t-1}
\end{align*}

We see that in this case, the trend has been completely removed, $at$.

Perform differencing to eliminate the trend from the timeseries.

In [ ]:
def trend_removal_via_differencing(X, diff=1):
    plot_timeseries(X, 'Timeseries Data Before Differencing')
    
    #TODO: implement regular differencing
    #Perform differencing diff times
    
    #YOUR CODE HERE
    Y = ...
    for _ in range(diff):
        
    #END YOUR CODE
        
    plot_timeseries(Y, 'Timeseries Data After Differencing')
    plt.legend()
    plt.title("Timeseries data after differencing %d times" % diff)
    plt.show()
    
    
X_linear_trend = generate_timeseries(deg='linear')
X_quad_trend = generate_timeseries(deg='quadratic')
X_sin_trend = generate_timeseries(deg='sinusoidal')

trend_removal_via_differencing(X_linear_trend)
trend_removal_via_differencing(X_quad_trend, diff = 2)
trend_removal_via_differencing(X_sin_trend, diff=1)

#using differencing on Tesla data
trend_removal_via_differencing(price)

### Observations:

1. Which data did differencing perform well on?
2. Which data did differencing perform poorly on?
3. For linear data, what level of differencing should be used?
4. For quadratic data, what level of differencing should be used? (Try the math)

### Question 2: Seasonal Differencing

You probably saw that differencing performed poorly on some of the data because it had a seasonal trend.

If our data has a seasonal trend, meaning it can be expressed as:

\begin{align*}
X_{t} = m_{t} + Z_{t}\\
m_{t} = m_{t-p}
\end{align*}

where $p$ is the period of the seasonal trend.

Seasonal differencing is a technique in which we can create a new timeseries:

\begin{align*}
Y_{t} = X_{t} - X_{t-p}
\end{align*}

If our trend is linear, for instance, we can achieve the following result using differencing:

\begin{align*}
m_{t} = m_{t-p}\\
X_{t} = m_{t} + Z_{t}\\
X_{t-p} = m_{t-p} + Z_{t-p}\\
Y = X_{t} - X_{t-p} = Z_{t} - Z_{t-p}
\end{align*}

We see that in this case, the trend has been completely removed by using the property of our trend's seasonality.

In [ ]:
def trend_removal_via_seasonal_differencing(X, diff=1, period = 1):
    plot_timeseries(X, 'Timeseries Data Before Differencing')
    
    #TODO: implement seasonal differencing
    #Perform differencing diff times
    
    #YOUR CODE HERE
    Y = ... #output
    for _ in range(diff):
        
    #END YOUR CODE
        
    plot_timeseries(Y, 'Timeseries Data After Seasonal Differencing')
    plt.legend()
    plt.title("Timeseries data after seasonal differencing %d times" % diff)
    plt.show()
    
    
X_linear_trend = generate_timeseries(deg='linear')
X_quad_trend = generate_timeseries(deg='quadratic')
X_sin_trend = generate_timeseries(deg='sinusoidal')

trend_removal_via_seasonal_differencing(X_linear_trend)
trend_removal_via_seasonal_differencing(X_quad_trend)
trend_removal_via_seasonal_differencing(X_sin_trend, period=20)

#using differencing on Tesla data
trend_removal_via_seasonal_differencing(price)

### Observations:
1. Did the timeseries that trend differencing failed on work with seasonal differencing? With what parameter period?
2. How does seasonal differencing work on linear/quadratic data? Derive what seasonal differencing does to data that is not actually seasonal. You should see that it effectively removes the trend while introducing more bias.

### Question 3:

Performing both seasonal and trend differencing on data can remove both seasonal and non-seasonal trends.

If our data has a seasonal and non-seasonal trend, meaning it can be expressed as:

\begin{align*}
X_{t} = m_{t} + s_{t} + Z_{t}\\
s_{t} = s_{t-p}
\end{align*}

where $p$ is the period of the seasonal trend. $m_{t}$ is the non-seasonal trend.

Combining both seasonal and non-seasonal differencing is a technique in which we can create a new timeseries:  
First, seasonal differencing:
\begin{align*}
Y_{t} = m_{t} - m_{t-p} + Z_{t} - Z_{t-p}
\end{align*}

Then, regular differencing:
\begin{align*}
Y_{t} = m_{t} - m_{t-p} + Z_{t} - Z_{t-p}\\
Y_{t-1} = m_{t-1} - m_{t-p-1} + Z_{t-1} - Z_{t-p-1}\\
Y_{t} - Y_{t-1} = c + Z_{t} - Z_{t-p} + Z_{t-1} - Z_{t-p-1}
\end{align*}

We see that by performing first seasonal differencing, and then regular differencing, we can remove both a seasonal and trend component from the data.

In [ ]:
def trend_removal_via_seasonal_nonseasonal_differencing(X, diff=1, period = 1, seasonal_diff=1):
    plot_timeseries(X, 'Timeseries Data Before Differencing')
    
    #TODO: Implement seasonal differencing first seasonal_diff times, then perform regular differencing diff times
    #YOUR CODE HERE
    Y = ... #output
    for _ in range(seasonal_diff):
        
    for _ in range(regular_diff):
    
    #END YOUR CODE
        
    plot_timeseries(Y, 'Timeseries Data After Both Differencing')
    plt.legend()
    plt.title("Timeseries data after both differencing: %d times regular, %d times seasonal" % (diff, seasonal_diff))
    plt.show() 
    
X_linear_trend = generate_timeseries(deg='linear')
X_seasonal_trend = generate_timeseries(deg='sinusoidal')

X_both = X_linear_trend + X_seasonal_trend

trend_removal_via_seasonal_nonseasonal_differencing(X_both)

### Observations
1. Did it work? Does the data now look stationary, there is no underlying direction to the data?

## Part 3: ARIMA using statsmodels

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from statsmodels.tsa.statespace.sarimax import SARIMAX

import datetime as dt

In [ ]:
data = pd.read_csv('data/TSLA.csv')
time = np.array(data['Date'])
price = np.array(data['Close'])

plt.title('Tesla Stock Price')
plt.plot(time, price)
plt.xticks(time[::50])
plt.xlabel('Days since November 19, 2019')
plt.ylabel('Price')
plt.show()

The graph above depicts the stock price of TSLA over time. The x-axis being time and the y-axis being f(t), the price at that point in time.

Clearly a basic linear model wouldn't be able to model such a function of time due to its non-linearity. However, we have previously learned ways to fit non-linear functions with linear regression using feature lifting. Maybe lifting some polynomial features might help?

### Trying Linear Regression

Linear features, linear regression

In [ ]:
time_steps = np.arange(0, time.shape[0], 1).astype(np.int).reshape((time.shape[0], 1))

#TODO: run a least squares linear regression in the format Ax=b, A = time_steps, B = price
#place the result in w
#Hint: try np.linalg.lstsq

#YOUR CODE HERE
w = None
#END YOUR CODE

plt.plot(time_steps, w * time_steps, label='linear fit')
plt.plot(time_steps, price, label = 'original data')
plt.legend()
plt.title('Tesla Stock Price with linear fit')
plt.xlabel('Days since November 19, 2019')
plt.ylabel('Price')
plt.show()


Polynomial features, linear regression

In [ ]:
def feature_lift(X, d=2):
    """
    This function computes second order variables
    for polynomial regression.
    Input:
    X: Independent variables.
    Output:
    A data matrix composed of both first and second order terms.
    """
    
    X = np.array(X)
    
    X_data = []
    
    for deg in range(1, d+1):
        X_data.append(np.power(X, deg))
        
    return np.hstack(X_data)

second_order_time_steps = feature_lift(time_steps, d=2)
w_2, err_2, _, _ = np.linalg.lstsq(second_order_time_steps[:210], price[:210])

third_order_time_steps = feature_lift(time_steps, d=3)
w_3, err_3, _, _ = np.linalg.lstsq(third_order_time_steps[:220], price[:220])

fifty_order_time_steps = feature_lift(time_steps, d=50)
w_50, err_50, _, _ = np.linalg.lstsq(fifty_order_time_steps[:220], price[:220])

plt.plot(time_steps, second_order_time_steps @ w_2.T, label='deg 2 fit')
plt.plot(time_steps, third_order_time_steps @ w_3.T, label='deg 3 fit')
plt.plot(time_steps, fifty_order_time_steps @ w_50.T, label='deg 50 fit')
plt.plot(time_steps, price, label = 'original data')

plt.xlim(right=250)
plt.xlabel('Days since November 19, 2019')
plt.ylabel('Price')
plt.legend()
plt.title('Tesla stock price with linear regression on feature lift')
plt.show()

print('Training error with degree 2 features: %s' % str(err_2))
print('Training error with degree 3 features: %s' % str(err_3))
print('Training error with degree 50 features: %s' % str(err_50))

Basically, any attempt to fit some sort of function using linear regression and feature lifting will be met with failure. Why? This is due to the nature of time series data. The problem that linear regressions try to solve is fitting some function to the data in its entirety. However, time series data, such as stock prices don't operate under the same assumptions. Does the stock price 1 year ago directly affect the stock price tomorrow? The answer is, not really. This understanding that only the previous n' << n observations are truly important to our model spawns this set of models that utilize the moving average, the first MA model, then the ARMA, and ARIMA models.

### Formulation of the ARIMA Model

The mathematical model behind ARIMA is:

\begin{align*}
ARIMA(p, d, q) = (1 - \phi_{1}L - \phi_{2}L^{2} - \phi_{3}L^{3}... - \phi_{p}L^{p}) (1 - L)^{d} y_{t} = c + (1 + \theta_{1} L + \theta_{2} L^{2}... + \theta_{q} L^{q}) \epsilon_{t}
\end{align*}

The L is the lag operator, $L^d y_{t} = y_{t - d}$.

The first group of terms, containing $\phi$'s, are the auto-regressive terms. Auto-regressive refers to the contribution of previous observations on the current observation. The parameter $p$ is the number of previous observations looked at. The $\phi$'s are their individual weighting. Intuitively, an observation 1 time step ago usually holds more weight than an observation 10 time steps ago.

The second term, $(1 - L)^{d}$, is the integrated term. This is the term that implements the idea of differencing, which is sometimes necessary to find a deeper relationship in the data. The time series function may not be able to model the observation themselves, but may be able to model the difference between successive observations.

The third and final term, containing $\theta$'s, represent the moving average portion of the model. This is the contribution of the previous $q$ terms, specifically their residuals to the residual at time t. The residuals, $\epsilon_{t}$, are standard normal distribution deviations of each observation from the predicted observation predicted by the model. These residuals are what we try to minimize when fitting parameters to the training data.

The constant c quantifies the drift of the model.

## Breaking Down the ARIMA Model

### Auto-Regressive (AR)

One of the fundamental building blocks of the ARIMA model is the AR portion of the model. The equation for a simple AR model is:

\begin{align*}
AR(p) = y_{t} = c + (\phi_{1}L + \phi_{2}L^{2} + \phi_{3}L^{3}... + \phi_{p}L^{p})y_{t} + \epsilon_{t}
\end{align*}

This equation can already begin to model functions under the assumption that current observations are simply derived from a linear combination of the p previous observations.



For the rest of this part of the notebook, we'll be working with weather data instead of stock data.

In [ ]:
#load dataset
weather_data = pd.read_csv('data/weather_data.csv', parse_dates=['datetime_utc'], index_col='datetime_utc')
weather_data = weather_data.rename(index=str, columns={' _tempm': 'temperature'})

#interpolate null values
weather_data.ffill(inplace=True)
weather_data.index = pd.to_datetime(weather_data.index)

#remove outliers
weather_data = weather_data[weather_data.temperature < 50]
weather_temp = weather_data['temperature']

weather_temp.plot()
plt.xlabel('time')
plt.ylabel('temp in deg C')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Historical weather data')
plt.show()

print(weather_temp.head())
print('Null values: %s' % str(weather_temp.isnull().sum()))

The first thing we do with our data is clean it, interpolating null values and making sure the samples are evenly spaced. Next, there are some things we can check about the data itself to ensure it is suitable for an ARIMA model.

In [ ]:
training_data = weather_temp['2000':'2014'].resample('M').mean().fillna(method='pad')
testing_data = weather_temp['2015':'2017'].resample('M').mean().fillna(method='pad')
training_data.plot()
testing_data.plot()
plt.title('Training and Testing Data Split')
plt.show()

In the next cell we will plot the autocorrelation of both the original data and the differences between consecutive data points. There is also a provided function ``plot_rolling_mean_std``. Try to find a period length that creates a constant rolling mean. What does this say about the periodicity of the data?

In [ ]:
# check rolling mean and rolling standard deviation
def plot_rolling_mean_std(ts, period):
    rolling_mean = ts.rolling(period).mean()
    rolling_std = ts.rolling(period).std()

    plt.plot(ts, label='Actual Mean')
    plt.plot(rolling_mean, label='Rolling Mean')
    plt.plot(rolling_std, label = 'Rolling Std')
    plt.xlabel("Date")
    plt.ylabel("Mean Temperature")
    plt.title('Rolling Mean & Rolling Standard Deviation')
    plt.legend()
    plt.show()

#TODO: change this period parameter until the resulting rolling mean is 0 (Hint: What do you think the period of this data is?)
plot_rolling_mean_std(training_data, period = 1)
    
plt.plot(training_data)
plot_acf(training_data)
plt.show()

plt.plot(training_data.diff())
plot_acf(training_data.diff().values)
plt.show()


We see from the rolling mean and rolling standard deviation plot that the data has a constant mean and standard deviation on a 12 month period. This periodicity implies that our data is seasonal, meaning a standard ARIMA model will have trouble fitting it, but instead we will need to use the seasonal aspect of SARIMA to fit the data.

First, we will try to see what kind of model we retrieve from an AR model.

In [ ]:
def process_result(res, model='arima'):
    print(res.summary())
    
    plt.plot(res.resid)
    plt.title('Training Residuals')
    plt.xlabel('Months')
    plt.ylabel('Temperature in C')
    plt.show()

    print('Mean squared training error: %s' % str(np.square(res.resid).mean()))

    plt.plot(res.predict(), label='Prediction')
    plt.plot(training_data.values, label='Real values')
    plt.legend()
    plt.title('Model fit on training data')
    plt.xlabel('Months')
    plt.ylabel('Temperature in C')
    plt.show()
    
    if model == 'arima':
        fc, se, conf = res.forecast(testing_data.shape[0], alpha=0.05)
    elif model == 'sarimax':
        fc = res.forecast(testing_data.shape[0], alpha=0.05)
        
    fc_series = pd.Series(fc, index=testing_data.index)
    plt.plot(fc_series, label='Prediction')
    plt.plot(testing_data, label='Real values')
    plt.legend()
    plt.title('Model prediction on test data')
    plt.xlabel('Datetime')
    plt.ylabel('Temperature in C')
    plt.show()
    
    #TODO: Calculate the mean squared testing error
    #fc = prediction, testing_data = true observations
    squared_testing_error = 0
    print('Mean squared testing error: %s' % str(squared_testing_error))
    

In [ ]:
#TODO: use ARIMA(data, order).fit() to generate an AR model fit using order=(p, 0, 0)
#https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARIMA.html
#Hint: try p > 1
res = ...
process_result(res)


### Write down your observations.

How does the model perform on the training data?

How does the model perform on the forecasting of future data?

Why does the AR model not fit this seasonal data well?

### Moving Average (MA)

Another fundamental building block of the ARIMA model is the MA portion of the model. The equation for a simple MA model is:

\begin{align*}
MA(q) = y_{t} = c + \epsilon_{t} + \theta_{1}\epsilon_{t-1} + \theta_{2}\epsilon_{t-2} + ... + \theta_{q}\epsilon_{t-q}
\end{align*}

This equation models the assumption that the residual, the deviation from the predicted point, of the current observation is a linear combination of the previous q residuals.

Add the moving average section of the ARIMA model, and model the weather data using an ARMA model.

In [ ]:
#TODO: use ARIMA(data, order).fit() to generate an ARMA model fit using order=(p, 0, q)
#https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARIMA.html
#Hint: try p, q > 1
res = ...
process_result(res)

### Write down your observations.

How does the model perform on the training data?

How does the model perform on the forecasting of future data?

Does the ARMA model fit the seasonal data better than the AR model?

## Seasonality in ARIMA: SARIMA

As we noted in the weather data, we see a seasonal pattern, or period with fixed period length. The traditional ARIMA model is not fully suited to model this behavior, but this is where SARIMA comes into play. SARIMA, season auto-regressive integrated moving average, has a slightly different formulation from ARIMA:

\begin{align*}
SARIMA(p, d, q) (P, D, Q)_{m} = (1 - \phi_{1}L - \phi_{2}L^{2} - \phi_{3}L^{3}... - \phi_{p}L^{p}) (1 - \Phi_{1}L^{m} - \Phi_{2}L^{2m} - \Phi_{3}L^{3m}... - \Phi_{P}L^{Pm}) (1 - L)^{d} (1 - L)^{Dm} y_{t} = c + (1 + \theta_{1} L + \theta_{2} L^{2}... + \theta_{q} L^{q}) (1 + \Theta_{1} L^{m} + \Theta_{2} L^{2m}... + \Theta_{Q} L^{Qm}) \epsilon_{t}
\end{align*}

The seasonal terms are terms that operate much like the original ARIMA terms, but the lag operators are all raised to the power of $m$, in the case of the annual cycle of weather data, $m=12$, meaning the model takes into account observations from 12, 24, 36.... months ago.

Implement a SARIMA model on the weather data.

In [ ]:
#TODO: use SARIMAX(data, order, seasonal_order).fit()
#https://www.statsmodels.org/stable/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.fit.html
#try p, q, P, Q > 1
res = ...
process_result(res, 'sarimax')

### Write down your observations.

How does the model perform on the training data?

How does the model perform on the forecasting of future data?

Does the SARIMAX model fit the seasonal data better than the ARIMA model?

## Part 4: Evaluating your ARIMA model

### Hyperparameters

You may have noticed that sometimes the performance is better than other times when using different values for p, d, q, P, D, and Q. These values are called hyperparameters, hand selected values by the engineer creating the model that greatly influence the effectiveness of the model. There are several ways to set these hyperparameters, as discussed earlier in this course. You can employ techniques such as k-fold cross validation and grid searches, as well as use domain knowledge to make educated guesses on what we want the model to learn and what the underlying model in reality is. For example, someone with strong knowledge of the weather can make a better educated guess about how many days one should look back in the past to predict today's weather.

In [ ]:
#TODO run a SARIMAX model with your guess
res = ...
process_result(res, 'sarimax')

Now, let's try using a grid search to find the best parameters.

In [ ]:
def get_test_mse(res):
    #TODO calculate test mse
    #use res.forecast(testing_data.shape[0]) to get predictions
    
    #YOUR CODE HERE
    fc = ...
    squared_testing_error = ...
    
    #END YOUR CODE
    return squared_testing_error

p_range = list(range(3))
q_range = list(range(3))
P_range = list(range(3))
Q_range = list(range(3))

best_params = (-1, -1, -1, -1)
best_test_mse = 10000000

#TODO: use a grid search to find the best parameters
for p in p_range:
    for q in q_range:
        for P in P_range:
            for Q in Q_range:
                #TODO: run SARIMAX on training_data.values, order = (p, 0, q), seasonal_order = (P, 0, Q, 12)
                #call your function get_test_mse(sarimax result)
                #save if new best test mse
                
                #YOUR CODE HERE
                
                #END YOUR CODE
                
                
print('Best parameters: ', best_params, ' Best Test MSE: ', best_test_mse)
res = SARIMAX(training_data.values, order=(best_params[0], 0, best_params[1]), seasonal_order=(best_params[2], 0, best_params[3], 12)).fit()
process_result(res, 'sarimax')

That probably took a while, was it worth it?

### Observations

How did the grid search perform compared to your guess?

### Evaluating our Model: AIC

Not only do we need to tune the hyperparameters to increase the performance of our model, but we also need a way to evaluate our model. We need a measure that maximizes the accuracy of the model while minimizing the number of parameters of our model, to avoid overfitting our training data. This is where AIC comes in. AIC is short for Akaike information criterion. It is formulated as follows:

\begin{align*}
AIC = -2log(L) + 2(p + q + k)
\end{align*}

k = 1 if the model includes an intercept, else k = 0.  
L = likelihood of our data (this depends on the residuals and dist. of white noise)

We want our model to minimize this quantity.  
Intuitively, we want to maximize the likelihiood of our data while our model is penalized by the complexity of the data.

Perform the grid search again on the hyperparameters while instead minimizing AIC instead of test_mse.

In [ ]:
p_range = list(range(3))
q_range = list(range(3))
P_range = list(range(2))
Q_range = list(range(2))

best_params = (-1, -1, -1, -1)
best_aic = 10000000

#TODO: use a grid search to find the best parameters
for p in p_range:
    for q in q_range:
        for P in P_range:
            for Q in Q_range:
                #TODO: run SARIMAX on training_data.values, order = (p, 0, q), seasonal_order = (P, 0, Q, 12)
                #save if new best aic
                
                #YOUR CODE HERE
                
                #END YOUR CODE
                
                
print('Best parameters: ', best_params, ' Best AIC: ', best_aic)
res = SARIMAX(training_data.values, order=(best_params[0], 0, best_params[1]), seasonal_order=(best_params[2], 0, best_params[3], 12)).fit()
process_result(res, 'sarimax')

### Observations

1. Did the best test MSE model differ from the best AIC model?
2. Which one had smaller parameters?

## Part 5: Implementing an AR model solver

As we noted in the notes, implementing an AR solver is much easier than an MA solver. This is because an AR solver models the each observation as a linear combination of $p$ previous observations, while an MA solver involves residuals which comes as a result of the model itself, requiring more complicated solving methods. Interestingly, we found in the notes in the section on invertibility that oftentimes MA models can be recast as an AR model. Refer to section 6.1 of the notes for further reading.

In the following part, we will implement an AR solver ourselves.

First, load the dataset inside data/weather_data.csv, like Part 3 of the notebook. We will build an AR model solver and test it on this dataset, but using a different observation than temperature.

In [ ]:
#load dataset
weather_data = pd.read_csv('data/weather_data.csv', parse_dates=['datetime_utc'], index_col='datetime_utc')
weather_data = weather_data.rename(index=str, columns={' _tempm': 'temperature', ' _hum': 'humidity', ' _dewptm': 'dew_point_temp', ' _pressurem': 'pressure'})
weather_data.index = pd.to_datetime(weather_data.index)

weather_data.head()

Select a column from weather_data and create a new dataframe from weather_data with only that column.  
Note: the keys in the dataframe may have some whitespace so make sure you are spelling everything correctly to index the column.  

Then, we need to interpolate null values and remove outliers (clean the data).  
Refer to the code blocks in Part 3 or check out Pandas documentation.

In [ ]:
#TODO: Choose a column from weather_data (solution uses humidity)

#YOUR CODE HERE
weather_data_one_col = ...

#TODO: interpolate null values
#reference the appropriate code section from Part 3


#TODO: remove outliers
#reference the appropriate code section from Part 3
weather_data_one_col = ...

#END YOUR CODE

weather_data_one_col.plot()
plt.xlabel('time')
plt.ylabel('Humidity')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Historical weather data')
plt.show()

print(weather_data_one_col.head())
print('Null values: %s' % str(weather_data_one_col.isnull().sum()))

Now, we need to resample the data to regular intervals as well as split the data into training and testing data.

In [ ]:
training_data = weather_data_one_col['2000':'2014'].resample('M').mean().fillna(method='pad')
testing_data = weather_data_one_col['2015':'2017'].resample('M').mean().fillna(method='pad')
training_data.plot()
testing_data.plot()

Now fill out the AR_Model class.

In [ ]:
class AR_Model():
    def __init__(self, q):
        #TODO: initialize variables, for the AR Model (q, phi's)
        #Think: How many phi's do we need
        
        #YOUR CODE HERE
        self.phi = ...
        #END YOUR CODE
        
    def fit(self, data):
        #TODO: fit parameters to the data
        #steps:
        #create a data matrix (A), each row should have q previous observations
        #create a label vector (b), each element should be one observation
        #solve least squares Ax = b, x = phi's
        
        #return a vector of observations predicted from the fitted model, the same shape as data
        #multiply data matrix against the predicted model phi's and return the output vector
        
        #need to keep this for predicting
        self.data = data
        
        #YOUR CODE HERE
        data_mat = []
        label_vector = []
        for i in range(data.shape[0] - self.phi.shape[0]):
            data_mat.append(...)
            label_vector.append(...)
            
        data_mat = np.array(data_mat)
        label_vector = np.array(label_vector)
        
        w = ...
        self.phi = ...
        
        #don't predict anything for first q observations as there are not q previous observations for the AR Model
        predicted = np.zeros(self.phi.shape[0])
        return ...
        #END YOUR CODE
    
        
    def predict(self, n):
        #TODO: predict the next n observations given the fitted model
        
        #YOUR CODE HERE
        assert(len(self.data) > 0)
        data = list(np.copy(self.data))
        
        predictions = []
        for i in range(n):
            next_prediction = ...
            data.append(next_prediction)
            predictions.append(next_prediction)
            
        return np.array(predictions)
        
        #END YOUR CODE
        

Edit and Run the following code to test your model and plot against both training and testing values.

In [ ]:
q = 10
arm = AR_Model(q)

model_fit_training = arm.fit(training_data)

plt.plot(training_data.values, label='training data')
plt.plot(model_fit_training, label='model fit')
plt.title('Student implemented AR Model with parameter %d' % q)
plt.xlabel('Time')
plt.ylabel('Observations')
plt.legend()
plt.show()


predicted = arm.predict(testing_data.shape[0])

plt.plot(testing_data.values, label='testing data')
plt.plot(predicted, label='model predicted')
plt.title('Student implemented AR Model predictions with parameter %d' % q)
plt.xlabel('Time')
plt.ylabel('Observations')
plt.legend()
plt.show()

Now, run the statsmodels implementation of an AR model on the same data. Yours should look similar.

In [ ]:
res = ARIMA(training_data.values, order=(q, 0, 0)).fit()
process_result(res)

Your code should run much faster than the statsmodels package. This is because we are implementing a very simple solver for the AR model parameters. If you would like to find out more about more complex and precise solvers, such as Maximum Likelihood Estimators, check out statsmodels opensource code hosted on github: https://github.com/statsmodels/statsmodels. Although we only wrote an AR solver in this part of the code, we derived the invertibility of MA models to AR models in the notes, so this solver could be applicable to any ARMA model. Supporting ARIMA models would require simply adding a differencing step.

## Part 6: Cool ARIMA models

There are some cool patterns that can be modeled using ARIMA. 
Do not use statsmodels.

1. Random Walk: ARIMA(0, 1, 0)
2. Random Walk with drift: ARIMA(0, 1, 0) w/ a constant


First: Derive the explicit formulation of each cool pattern as $X_t = $:

Random Walk:

Random Walk with drift:

Implement the following functions.

In [ ]:
def random_walk(num_steps=100):
    state = 0
    ts = []
    
    #TODO: fill ts with num_steps predictions from random_walk model
    #use np.random.normal(0, 1) for Zt
    
    #YOUR CODE HERE
    for i in range(num_steps):
        ts.append(0)
        
    #END YOUR CODE
        
    return ts

def random_walk_with_drift(num_steps=100, drift=0.4):
    state = 0
    ts = []
    
    #TODO: fill ts with num_steps predictions from random_walk model
    #use np.random.normal(0, 1) for Zt
    
    #YOUR CODE HERE
    for i in range(num_steps):
        ts.append(0)
        
    #END YOUR CODE
        
    return ts
    

random_walk_ts = random_walk()
random_walk_drift_ts = random_walk_with_drift()
dampled_holts_ts = dampled_holts()

plt.plot(random_walk_ts)
plt.title("Random Walk")
plt.xlabel("time")
plt.ylabel("observation")
plt.show()

plt.plot(random_walk_drift_ts)
plt.title("Random Walk with drift")
plt.xlabel("time")
plt.ylabel("observation")
plt.show()

### Observations

1. Run the above cell a few times. What does the Random Walk look like? What does the Random Walk with drift look like?

# Conclusions

This notebook provides an introduction to time series modeling, first exploring the concepts behind smoothing and stationarity, then using the open source statsmodels library to generate and fit ARIMA models, evaluating our ARIMA models, and finally exploring different patterns that can be modeled using ARIMA.